In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from time import time

# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat

In [2]:
#Reading the CSV file
df = pd.read_csv("Datasets/cleaned_weatherAUS.csv")

In [3]:
Direction = ['N','NNE','NE','ENE','E','ESE','SE','SSE','S','SSW','SW','WSW','W','WNW','NW','NNW']
Degree = []

x = 0
for i in range(0,337,22):
    Degree.append(i+x)
    x = x + 0.5
    
Direction_Degree = dict(zip(Direction,Degree))
print(Direction_Degree)

#Converting WindGustDir, WindDir9am, WindDir3pm to degree format

df = df.replace({"WindGustDir": Direction_Degree})
df = df.replace({"WindDir9am": Direction_Degree})
df = df.replace({"WindDir3pm": Direction_Degree})

{'N': 0, 'NNE': 22.5, 'NE': 45.0, 'ENE': 67.5, 'E': 90.0, 'ESE': 112.5, 'SE': 135.0, 'SSE': 157.5, 'S': 180.0, 'SSW': 202.5, 'SW': 225.0, 'WSW': 247.5, 'W': 270.0, 'WNW': 292.5, 'NW': 315.0, 'NNW': 337.5}


In [4]:
df = df.drop(columns = ['Date','Location','RainTomorrow'],axis=1)

In [5]:
df

,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,Month
0,1.0,16.8,0.0,112.5,24.0,112.5,90.0,9.0,9.0,82.0,45.0,1024.9,1020.2,7.0,16.0,No,9
1,6.1,20.7,0.0,45.0,43.0,135.0,45.0,4.0,24.0,80.0,54.0,1016.6,1010.8,10.2,20.3,No,9
2,6.3,16.9,1.4,315.0,35.0,112.5,292.5,6.0,13.0,85.0,58.0,1014.4,1012.3,12.5,15.5,Yes,9
3,2.1,15.0,0.0,292.5,30.0,112.5,315.0,4.0,11.0,81.0,54.0,1019.5,1017.9,8.6,14.9,No,9
4,1.6,16.6,0.0,45.0,37.0,135.0,45.0,9.0,20.0,79.0,47.0,1018.4,1011.4,8.3,15.4,No,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109518,3.5,21.8,0.0,90.0,31.0,112.5,90.0,15.0,13.0,59.0,27.0,1024.7,1021.2,9.4,20.9,No,6
109519,2.8,23.4,0.0,90.0,31.0,135.0,67.5,13.0,11.0,51.0,24.0,1024.6,1020.3,10.1,22.4,No,6
109520,3.6,25.3,0.0,337.5,22.0,135.0,0.0,13.0,9.0,56.0,21.0,1023.5,1019.1,10.9,24.5,No,6
109521,5.4,26.9,0.0,0.0,37.0,135.0,292.5,9.0,9.0,53.0,24.0,1021.0,1016.8,12.5,26.1,No,6


In [6]:
from sklearn.preprocessing import LabelEncoder

cat_f = [x for x in df.columns if df[x].dtype == 'object']
print(cat_f)

# traversing all columns for converting into continuous valriables
for name in cat_f:
    enc = LabelEncoder()
    enc.fit(list(df[name].values.astype('str')) + list(df[name].values.astype('str')))
    df[name] = enc.transform(df[name].values.astype('str'))

['RainToday']


In [7]:
df

,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,Month
0,1.0,16.8,0.0,112.5,24.0,112.5,90.0,9.0,9.0,82.0,45.0,1024.9,1020.2,7.0,16.0,0,9
1,6.1,20.7,0.0,45.0,43.0,135.0,45.0,4.0,24.0,80.0,54.0,1016.6,1010.8,10.2,20.3,0,9
2,6.3,16.9,1.4,315.0,35.0,112.5,292.5,6.0,13.0,85.0,58.0,1014.4,1012.3,12.5,15.5,1,9
3,2.1,15.0,0.0,292.5,30.0,112.5,315.0,4.0,11.0,81.0,54.0,1019.5,1017.9,8.6,14.9,0,9
4,1.6,16.6,0.0,45.0,37.0,135.0,45.0,9.0,20.0,79.0,47.0,1018.4,1011.4,8.3,15.4,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109518,3.5,21.8,0.0,90.0,31.0,112.5,90.0,15.0,13.0,59.0,27.0,1024.7,1021.2,9.4,20.9,0,6
109519,2.8,23.4,0.0,90.0,31.0,135.0,67.5,13.0,11.0,51.0,24.0,1024.6,1020.3,10.1,22.4,0,6
109520,3.6,25.3,0.0,337.5,22.0,135.0,0.0,13.0,9.0,56.0,21.0,1023.5,1019.1,10.9,24.5,0,6
109521,5.4,26.9,0.0,0.0,37.0,135.0,292.5,9.0,9.0,53.0,24.0,1021.0,1016.8,12.5,26.1,0,6


In [8]:
# Dividing Data into features and target for classifier
X = df.drop(['Rainfall'], axis=1)
Y = df['Rainfall']

In [9]:
# our test dataset doesn't have a target variable, so we'll have to test on the train df using holdout 
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [10]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [11]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train, y_train)

'from sklearn.linear_model import LinearRegression\nregressor = LinearRegression()\nregressor.fit(x_train, y_train)'

In [12]:
results = regressor.predict(x_test)

'results = regressor.predict(x_test)'

In [13]:
print(mean_squared_error(y_test, results))
print(mean_absolute_error(y_test, results))
print(r2_score(y_test,results))

'print(mean_squared_error(y_test, results))\nprint(mean_absolute_error(y_test, results))\nprint(r2_score(y_test,results))'